# Get GAV estimation for resonance radical

In [1]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import random
import os
import time
import math
from copy import deepcopy
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

from rmgpy.data.thermo import ThermoDatabase, ThermoLibrary, ThermoData, remove_thermo_data, add_thermo_data, NASA
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy import settings
from rmgpy import constants


from tree.utils import load_thermo_lib_by_path, generate_thermo
from tree.parameters import Ts

# Load resonance radicals

In [2]:
resonance_radical_df = pd.read_csv("../data/resonance_radicals.csv")
resonance_radical_df

,radical_smiles,resonance_radical_smiles,has_H,has_C,has_N,has_O,Mw (g/mol),num_rings,num_heavy_atoms,num_rotatable_bonds,radical_atom_type
0,[O]C(=O)OC(O)(O)O,[O]C(=O)OC(O)(O)O,True,True,False,True,122.992963,0,8,4,O
1,[O]C(O)(O)OC(=O)O,[O]C(O)(O)OC(=O)O,True,True,False,True,122.992963,0,8,3,O
2,O=C(O)O[C](O)O,O=C(O)O[C](O)O,True,True,False,True,106.998048,0,7,3,C
3,CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)O[O],True,True,False,True,195.014092,0,13,8,O
4,O=[C]OC(O)(O)O,O=[C]OC(O)(O)O,True,True,False,True,106.998048,0,7,5,C
...,...,...,...,...,...,...,...,...,...,...,...
2804,C1=C[CH]C=1,[C]1=CC=C1,True,True,False,False,51.023475,1,4,0,C
2805,C=C=C1C=[C]C1,C=C=C1C=[C]C1,True,True,False,False,77.039125,1,6,0,C
2806,[CH2]C1=CC#CC1,[CH2]C1=CC#CC1,True,True,False,False,77.039125,1,6,1,C
2807,[CH2]C1=CC#CC1,C=C1[CH]C#CC1,True,True,False,False,77.039125,1,6,0,C


# Estimate GAV for resonance radicals

In [3]:
# load thermo database

thermo_database = ThermoDatabase()
thermo_database.load_groups(os.path.join(settings["database.directory"], "thermo", "groups"))

In [4]:
generate_thermo(thermo_database, "[O]C(=O)OC(O)(O)O", resonance=False) # we want to know the GAV estimation for the exact resonance radical

/home/gridsan/hwpang/Software/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: No module named 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


(-214.5628470363289,
 98.46305594224867,
 array([29.53076482, 33.69692161, 37.1998088 , 39.86061185, 43.9456979 ,
        46.70558317, 51.10089866]),
 102.02365200764818,
 'Thermo group additivity estimation: group(O2s-Cs(Cds-O2d)) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-(Cds-O2d)H) + group(Cs-OsOsOsOs) + group(Cds-OdOsOs) + radical(OC=OOJ)')

In [5]:
resonance_radical_gav_df = pd.DataFrame()
resonance_radical_gav_df["smiles"] = resonance_radical_df["resonance_radical_smiles"]

In [6]:
thermos = resonance_radical_gav_df["smiles"].apply(lambda smi: generate_thermo(thermo_database, smi, resonance=False))

In [7]:
temp_columns = ["H298 (kcal/mol)", "S298 (cal/mol/K)", "Cp (cal/mol/K)", "Sint298 (cal/mol/K)", "comment"]
resonance_radical_gav_df[temp_columns] = thermos.to_list()

temp_columns = []
for i, T in enumerate(Ts):
    temp_columns.append(f"Cp{T} (cal/mol/K)")
resonance_radical_gav_df[temp_columns] = resonance_radical_gav_df["Cp (cal/mol/K)"].to_list()

resonance_radical_gav_df = resonance_radical_gav_df.drop("Cp (cal/mol/K)", axis=1)

temp_columns = resonance_radical_gav_df.columns.tolist()
temp_columns.remove("comment")
temp_columns.append("comment")

resonance_radical_gav_df = resonance_radical_gav_df[temp_columns]
resonance_radical_gav_df

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,smiles,H298 (kcal/mol),S298 (cal/mol/K),Sint298 (cal/mol/K),Cp300 (cal/mol/K),Cp400 (cal/mol/K),Cp500 (cal/mol/K),Cp600 (cal/mol/K),Cp800 (cal/mol/K),Cp1000 (cal/mol/K),Cp1500 (cal/mol/K),comment
0,[O]C(=O)OC(O)(O)O,-214.562847,98.463056,102.023652,29.530765,33.696922,37.199809,39.860612,43.945698,46.705583,51.100899,Thermo group additivity estimation: group(O2s-...
1,[O]C(O)(O)OC(=O)O,-218.506442,100.780822,100.780822,28.957151,32.645296,36.363289,39.621606,44.949522,48.354723,52.797839,Thermo group additivity estimation: group(O2s-...
2,O=C(O)O[C](O)O,-182.366843,95.359992,96.737419,26.444474,30.605583,34.122333,36.909560,40.899044,43.401300,45.798700,Thermo group additivity estimation: group(O2s-...
3,CC(=O)OC(OO)C(=O)C(O)O[O],-198.122312,145.818548,145.246864,50.656616,59.598776,67.892505,73.980363,82.723365,90.133078,97.599981,Thermo group additivity estimation: group(O2s-...
4,O=[C]OC(O)(O)O,-169.366862,91.656926,93.840096,27.460975,30.914895,33.801147,36.038910,39.770268,42.499082,46.915908,Thermo group additivity estimation: group(O2s-...
...,...,...,...,...,...,...,...,...,...,...,...,...
2804,[C]1=CC=C1,156.8305,62.382873,63.7603,15.164000,18.710000,21.534000,23.741000,26.989000,29.289000,32.814000,Thermo group additivity estimation: group(Cds-...
2805,C=C=C1C=[C]C1,151.763874,52.204958,53.582385,22.076919,28.279266,33.060461,36.665145,41.822263,45.330044,50.563950,Thermo group additivity estimation: group(Cs-(...
2806,[CH2]C1=CC#CC1,160.124,71.59,71.59,22.319000,28.397000,32.609000,37.198000,43.459000,47.876000,52.869000,Thermo group additivity estimation: group(Cs-(...
2807,C=C1[CH]C#CC1,183.114,79.63,79.63,21.319000,26.687000,31.089000,35.058000,41.669000,46.126000,50.759000,Thermo group additivity estimation: group(Cs-(...


In [8]:
resonance_radical_gav_df.to_csv("../data/resonance_radical_gav.csv", index=False)